In [311]:
import pandas as pd
import numpy as np
import psycopg2
import math
from datetime import timedelta

%matplotlib inline

con = psycopg2.connect (user = 'postgres',
                password = 'postgres',
                port = '5432',
                host = 'localhost',
                database = 'inicudb')

cur  = con.cursor()

In [312]:
uhidPatient = "('200700003','200700160','200700148','200700206','200700207')" 

#uhidPatient = "RSHI.0000012984"
caseType = 'Death'
#caseType = 'Discharge'
seperator = '_'

In [313]:
data = pd.read_csv('intermediate_checkpoint_apollo_5_9july.csv')

In [314]:
data['series'] = data.hour_series

In [315]:
data.columns

Index(['Unnamed: 0', 'actual_DOB', 'uhid', 'dischargeddate', 'actual_DOA',
       'gender', 'birthweight', 'birthlength', 'birthheadcircumference',
       'inout_patient_status', 'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'dischargestatus', 'birthlength.1',
       'birthheadcircumference.1', 'inout_patient_status.1',
       'gestationweekbylmp.1', 'gestationdaysbylmp.1', 'baby_type.1',
       'central_temp.1', 'apgar_onemin.1', 'apgar_fivemin.1', 'apgar_tenmin.1',
       'motherage.1', 'conception_type.1', 'mode_of_delivery.1',
       'steroidname.1', 'numberofdose.1', 'los', 'day_1', 'hour_series',
       'hour', 'day_x', 'abdomen_girth', 'urine', 'stool', 'stool_passed',
       'entry_timestamp', 'day_y', 'time_divide', 'series'],
      dtype='object')

In [316]:
data.urine.unique()


array([nan, 20., 30., 10., 60.,  8., 15., 25., 40.,  0., 12., 18., 22.,
       16., 32., 50.,  9., 24., 13., 11., 17., 23.])

In [317]:
print(data.uhid.unique())

[2.00700003e+08 2.00700148e+08 2.00700206e+08 2.00700207e+08
 2.00700160e+08            nan]


In [318]:
print(data)

       Unnamed: 0           actual_DOB         uhid       dischargeddate  \
0               0  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
1               1  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
2               2  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
3               3  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
4               4  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
5               5  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
6               6  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
7               7  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
8               8  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
9               9  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
10             10  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
11             11  2020-07-01 04:07:00  200700003.0  2020-07-09 23:00:00   
12          

In [319]:
data.time_divide.unique()

array([    0.   , 67926.231,  9161.544, 15695.336, 14177.385,  6158.858,
        5412.405, 17213.568,  6338.285,  5819.254,  7532.576, 21921.13 ,
        7262.306, 10701.935,  4991.719,  5019.998,  8797.234,  8147.022,
        4016.963, 19465.942,  2975.048,  8440.239, 13833.14 , 14833.706,
        6189.245,  9454.75 ,  7432.724,  6293.251, 15225.602, 13523.818,
        8113.174, 15612.621,  3659.78 , 31335.109,  5371.569,  7324.922,
       15107.769,  7507.39 , 20784.043,  7091.707,  9748.085,  7214.051,
        5138.189, 13349.124, 14592.792, 17490.681,  4777.271,  8791.391,
       12067.345,  9325.955, 12267.071, 13249.638,  7078.558,  8521.531,
       14758.633,  7044.721, 13417.279,  8694.653,  5025.666,  7759.91 ,
       10783.802,  6088.963,  5909.657,  7627.733,  7482.896, 57868.352,
       15691.813, 14334.069,  7929.152, 13956.866, 21476.22 ,  8133.036,
       19077.778,  6187.489, 15640.928, 15090.716,  6883.871,  6174.077,
       10189.675,  6642.52 , 20170.397, 24312.161, 

In [320]:
def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

def split_hour(x):
    try:
        t=str(x)
        return t.split(":")[0]
    except:
        pass
    
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass
    

In [321]:
def urine_check(x):
    try:
        if x > 200:
            return x/10
        else:
            return x
    except:
        pass

In [322]:
def to_float(x):
    if x == '':
        return np.nan
    else:
        return float(x)

In [323]:
dg = data.copy()

In [324]:
dg['urine'] = dg['urine'].apply(to_float)

In [325]:
dg['urine'] = dg['urine'].apply(urine_check)

In [326]:
dg['urine_per_hour'] = dg['urine']/((dg['time_divide'])/(3600))

In [327]:
dt = dg.sort_values(by = ['uhid','hour_series'])

In [328]:
ds = pd.DataFrame()

In [329]:
for i in dt.uhid.unique():
    x = dt[dt['uhid']==i]
    x['urine_per_hour'].fillna(method='bfill',inplace=True)
    ds = ds.append(x,ignore_index=True)
       

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [330]:
cur1 = con.cursor()
cur1.execute("SELECT uhid,creationtime,modificationtime,ph,entrydate FROM kalawati.nursing_bloodgas where uhid IN "+uhidPatient+";")
cols1 = list(map(lambda x: x[0],cur1.description))
ph = pd.DataFrame(cur1.fetchall(),columns=cols1)

In [331]:
ph

,uhid,creationtime,modificationtime,ph,entrydate


In [332]:
ph

,uhid,creationtime,modificationtime,ph,entrydate


In [333]:
def ph_func(x):
    try:
        p = float(x)
        if p>=6.80 and p<=7.8:
            return p
        else:
            return None
    except:
        pass

In [334]:
ph

,uhid,creationtime,modificationtime,ph,entrydate


In [335]:
ph['new_ph'] = ph.ph.apply(ph_func)

In [336]:
ph.dropna(subset=['new_ph'],inplace=True)

In [337]:
ph_df = pd.DataFrame()
for i in ph.uhid.unique():
    x = ph[ph['uhid']==i]
    ph_df = ph_df.append(x,ignore_index=True)

In [147]:
ph

,uhid,creationtime,modificationtime,ph,entrydate,new_ph


In [148]:
ph_df

""


In [149]:
#ph_df = pd.DataFrame(columns = ph.columns)


In [150]:
ph_df

""


In [338]:
ph_df_1 = None

In [339]:
if len(ph_df.columns)>0:
    ph_df['hour'] = ph_df.entrydate.apply(split_hour)
    ph_df.drop_duplicates(subset=['uhid','hour'],keep='last',inplace=True)
else:
    x = pd.DataFrame(data=[uhidPatient],columns={'uhid'})
    print(x)
    ph_df_1 = pd.DataFrame(columns=ph.columns)
    ph_df_1 = ph_df_1.append(x,ignore_index=True)

                                                uhid
0  ('200700003','200700160','200700148','20070020...


/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [340]:
if not(ph_df_1 is None) and len(ph_df_1.columns)>0 :
    ph_df_1

In [341]:
type(ds.uhid[0])

numpy.float64

In [342]:
ds['uhid'] = ds['uhid'].astype(str)


In [343]:
ds.uhid.unique()

array(['200700003.0', '200700148.0', '200700160.0', '200700206.0',
       '200700207.0'], dtype=object)

In [344]:
ph_df.uhid.unique()

AttributeError: 'DataFrame' object has no attribute 'uhid'

In [345]:
if len(ph_df.columns)>0:
    ss = pd.merge(ds,ph_df,on=['uhid','hour'],how='left')
    ss = ss.drop_duplicates(subset=['uhid','hour_series'],keep='first')
else:
    ss = pd.merge(ds,ph_df_1,on=['uhid'],how='left')
    ss = ss.drop_duplicates(subset=['uhid','hour_series'],keep='first')

In [346]:
ss.uhid.unique()

array(['200700003.0', '200700148.0', '200700160.0', '200700206.0',
       '200700207.0'], dtype=object)

In [158]:
ss

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,entry_timestamp,day_y,time_divide,series,urine_per_hour,creationtime,entrydate,modificationtime,new_ph,ph
0,0,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,0.0,2020-07-01 04:28:00,1.059973,NaN,NaN,NaN,NaN,NaN
1,1,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,0.0,2020-07-01 04:29:00,1.059973,NaN,NaN,NaN,NaN,NaN
2,2,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,0.0,2020-07-01 04:30:00,1.059973,NaN,NaN,NaN,NaN,NaN
3,3,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,0.0,2020-07-01 04:31:00,1.059973,NaN,NaN,NaN,NaN,NaN
4,4,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,0.0,2020-07-01 04:32:00,1.059973,NaN,NaN,NaN,NaN,NaN
5,5,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,0.0,2020-07-01 04:33:00,1.059973,NaN,NaN,NaN,NaN,NaN
6,6,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,0.0,2020-07-01 04:34:00,1.059973,NaN,NaN,NaN,NaN,NaN
7,7,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,0.0,2020-07-01 04:35:00,1.059973,NaN,NaN,NaN,NaN,NaN
8,8,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,0.0,2020-07-01 04:36:00,1.059973,NaN,NaN,NaN,NaN,NaN
9,9,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,0.0,2020-07-01 04:37:00,1.059973,NaN,NaN,NaN,NaN,NaN


In [347]:
len(ss.uhid.unique())

5

In [348]:
cur3 = con.cursor()
cur3.execute("SELECT t1.uhid,t1.entrydate,t1.rbs,t1.skintemp,t1.centraltemp  FROM kalawati.nursing_vitalparameters AS t1 where t1.uhid IN "+uhidPatient+";")
cols3 = list(map(lambda x: x[0], cur3.description))
nv = pd.DataFrame(cur3.fetchall(),columns=cols3)

In [349]:
nv.uhid.unique()

array(['200700003', '200700148', '200700206', '200700207', '200700160'],
      dtype=object)

In [350]:
nv.drop_duplicates(subset=['uhid','entrydate'],keep='first',inplace=True)

In [351]:
len(nv)

298

In [352]:
nv['hour'] = nv.entrydate.apply(split_hour)

In [164]:
nv.hour.iloc[0]

'2020-07-01 09'

In [356]:
nv['uhid'] = nv['uhid'].apply(to_float)

In [353]:
ss.uhid = ss.uhid.astype(str)

In [354]:
ss.uhid.unique()

array(['200700003.0', '200700148.0', '200700160.0', '200700206.0',
       '200700207.0'], dtype=object)

In [358]:
nv.uhid

0      200700003.0
1      200700003.0
2      200700003.0
3      200700003.0
4      200700003.0
5      200700003.0
6      200700003.0
7      200700003.0
8      200700003.0
9      200700003.0
10     200700003.0
11     200700003.0
12     200700003.0
13     200700003.0
14     200700003.0
15     200700003.0
16     200700003.0
17     200700003.0
18     200700003.0
19     200700003.0
20     200700003.0
21     200700003.0
22     200700003.0
23     200700003.0
24     200700003.0
25     200700148.0
26     200700206.0
27     200700003.0
28     200700148.0
29     200700003.0
          ...     
268    200700206.0
269    200700148.0
270    200700207.0
271    200700003.0
272    200700160.0
273    200700206.0
274    200700148.0
275    200700207.0
276    200700003.0
277    200700160.0
278    200700148.0
279    200700003.0
280    200700160.0
281    200700206.0
282    200700206.0
283    200700160.0
284    200700207.0
285    200700148.0
286    200700207.0
287    200700206.0
288    200700207.0
289    20070

In [359]:
ss.hour.iloc[0]

'2020-07-01 04'

In [361]:
ss['uhid'] = ss['uhid'].apply(to_float)

In [362]:
s1 = pd.merge(ss,nv,on = ['uhid','hour'], how='left')

In [168]:
len(ss)

41406

In [363]:
len(s1)

41946

In [364]:
len(s1.uhid.unique())

5

In [365]:
s1.drop_duplicates(subset=['uhid','hour_series'],keep='first',inplace=True)

In [172]:
s1

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,urine_per_hour,creationtime,entrydate_x,modificationtime,new_ph,ph,entrydate_y,rbs,skintemp,centraltemp
0,0,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,1.059973,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,1,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,1.059973,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,2,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,1.059973,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,3,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,1.059973,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,4,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,1.059973,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
5,5,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,1.059973,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
6,6,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,1.059973,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
7,7,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,1.059973,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
8,8,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,1.059973,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
9,9,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,1.059973,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [366]:
def temp_a(x,y):
    if not(x is None) and (x>0):
        return x
    else:
        return y

In [367]:
s1['temp'] = s1.apply(lambda x: temp_a(x['centraltemp'], x['skintemp']), axis=1)

In [368]:
cur2 = con.cursor()
cur2.execute("SELECT t1.uhid,t1.visitdate,t1.visittime,t1.currentdateweight, t1.currentdateheight  FROM kalawati.baby_visit AS t1 where t1.uhid  IN "+uhidPatient+";")
cols2 = list(map(lambda x: x[0], cur2.description))
bv = pd.DataFrame(cur2.fetchall(),columns=cols2)

In [369]:
bv

,uhid,visitdate,visittime,currentdateweight,currentdateheight
0,200700003,2020-07-02,06:35:34.977000,1340.0,NaN
1,200700003,2020-07-03,06:39:18.969000,1330.0,NaN
2,200700003,2020-07-04,06:42:51.876000,1350.0,NaN
3,200700003,2020-07-01,04:07:00,1390.0,39.0
4,200700148,2020-07-04,06:42:51.798000,1130.0,NaN
5,200700003,2020-07-06,06:29:43.684000,1410.0,NaN
6,200700206,2020-07-06,06:45:17.004000,1840.0,NaN
7,200700206,2020-07-07,06:57:22.244000,1840.0,NaN
8,200700148,2020-07-08,07:57:35.748000,1240.0,NaN
9,200700206,2020-07-04,18:33:00,1700.0,46.0


In [370]:
bv.sort_values('visitdate',inplace=True)

In [371]:
def stamp_1(x,y):
    return str(x) +" "+ str(y).split(":")[0]

In [372]:
bv

,uhid,visitdate,visittime,currentdateweight,currentdateheight
3,200700003,2020-07-01,04:07:00,1390.0,39.0
0,200700003,2020-07-02,06:35:34.977000,1340.0,NaN
1,200700003,2020-07-03,06:39:18.969000,1330.0,NaN
18,200700148,2020-07-03,13:40:00,1130.0,36.0
2,200700003,2020-07-04,06:42:51.876000,1350.0,NaN
4,200700148,2020-07-04,06:42:51.798000,1130.0,NaN
26,200700207,2020-07-04,18:35:00,1700.0,44.0
9,200700206,2020-07-04,18:33:00,1700.0,46.0
29,200700003,2020-07-05,06:43:00,1380.0,NaN
23,200700148,2020-07-05,06:33:00,1120.0,NaN


In [373]:
bv['hour'] = bv.apply(lambda x: stamp_1(x['visitdate'], x['visittime']), axis=1)

In [374]:
def to_str(x):
    return (str(x))

In [375]:
bv['day'] = bv['visitdate'].apply(to_str)

In [376]:
bv.drop_duplicates(subset=['uhid','visitdate'],keep='last',inplace=True)

In [377]:
s1['day'] = s1['hour_series'].apply(split_date_1)

In [378]:
s1.day.unique()

array(['2020-07-01', '2020-07-02', '2020-07-03', '2020-07-04',
       '2020-07-05', '2020-07-06', '2020-07-07', '2020-07-08',
       '2020-07-09'], dtype=object)

In [186]:
s1

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,entrydate_x,modificationtime,new_ph,ph,entrydate_y,rbs,skintemp,centraltemp,temp,day
0,0,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,NaN,NaN,NaT,NaN,None,None,None,2020-07-01
1,1,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,NaN,NaN,NaT,NaN,None,None,None,2020-07-01
2,2,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,NaN,NaN,NaT,NaN,None,None,None,2020-07-01
3,3,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,NaN,NaN,NaT,NaN,None,None,None,2020-07-01
4,4,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,NaN,NaN,NaT,NaN,None,None,None,2020-07-01
5,5,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,NaN,NaN,NaT,NaN,None,None,None,2020-07-01
6,6,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,NaN,NaN,NaT,NaN,None,None,None,2020-07-01
7,7,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,NaN,NaN,NaT,NaN,None,None,None,2020-07-01
8,8,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,NaN,NaN,NaT,NaN,None,None,None,2020-07-01
9,9,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,NaN,NaN,NaN,NaT,NaN,None,None,None,2020-07-01


In [382]:
s2 = pd.merge(s1,bv,on = ['uhid','day'], how='left')

In [381]:
bv['uhid'] = bv['uhid'].apply(to_float)

In [383]:
len(s2.uhid.unique())

5

In [189]:
#Meds

uhidPatient

"('200700003','200700160','200700148','200700206','200700207')"

In [384]:
cur7= con.cursor()
cur7.execute("SELECT t1.uhid,t1.startdate,t1.medicineorderdate,t1.medicinename,t1.medicationtype,t2.typevalue FROM kalawati.baby_prescription AS t1 LEFT JOIN kalawati.ref_medtype AS t2 ON t1.medicationtype = t2.typeid WHERE (t1.medicationtype = 'TYPE0001' OR t1.medicationtype = 'TYPE0009' OR t1.medicationtype = 'TYPE0004') and (t1.uhid IN " + uhidPatient+");")
cols7 = list(map(lambda x: x[0], cur7.description))
meds = pd.DataFrame(cur7.fetchall(),columns=cols7)

In [385]:
meds

,uhid,startdate,medicineorderdate,medicinename,medicationtype,typevalue
0,200700206,2020-07-05 12:55:36.990000+05:30,2020-07-05 12:55:36.990000+05:30,ZONAMAX-PS,TYPE0001,Antibiotics
1,200700148,2020-07-04 04:22:23.692000+05:30,2020-07-04 04:22:23.692000+05:30,Dopamine,TYPE0004,Inotropes
2,200700207,2020-07-05 12:43:25.410000+05:30,2020-07-05 12:43:25.410000+05:30,ZONAMAX-PS,TYPE0001,Antibiotics
3,200700148,2020-07-04 04:20:49.535000+05:30,2020-07-04 04:20:49.535000+05:30,ZONAMAX-PS,TYPE0001,Antibiotics
4,200700160,2020-07-07 00:18:05.437000+05:30,2020-07-07 00:18:05.437000+05:30,ZONAMAX-PS,TYPE0001,Antibiotics
5,200700206,2020-07-05 03:11:38.629000+05:30,2020-07-05 03:11:38.629000+05:30,Dopamine,TYPE0004,Inotropes


In [386]:
meds_dummies = pd.get_dummies(meds,columns=['typevalue'])


In [387]:
meds_dummies['hour'] = meds_dummies.startdate.apply(split_hour)


In [69]:
meds_dummies

,uhid,startdate,medicineorderdate,medicinename,medicationtype,hour


In [388]:
meds_dummies = pd.get_dummies(meds,columns=['typevalue'])
meds_dummies['hour'] = meds_dummies.startdate.apply(split_hour)
s2.drop(['hour_x','hour_y'],axis=1,inplace=True)

In [389]:
s2['hour'] = s2.hour_series.apply(split_hour)

In [196]:
meds_dummies

,uhid,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,typevalue_Inotropes,hour
0,200700206,2020-07-05 12:55:36.990000+05:30,2020-07-05 12:55:36.990000+05:30,ZONAMAX-PS,TYPE0001,1,0,2020-07-05 12
1,200700148,2020-07-04 04:22:23.692000+05:30,2020-07-04 04:22:23.692000+05:30,Dopamine,TYPE0004,0,1,2020-07-04 04
2,200700207,2020-07-05 12:43:25.410000+05:30,2020-07-05 12:43:25.410000+05:30,ZONAMAX-PS,TYPE0001,1,0,2020-07-05 12
3,200700148,2020-07-04 04:20:49.535000+05:30,2020-07-04 04:20:49.535000+05:30,ZONAMAX-PS,TYPE0001,1,0,2020-07-04 04
4,200700160,2020-07-07 00:18:05.437000+05:30,2020-07-07 00:18:05.437000+05:30,ZONAMAX-PS,TYPE0001,1,0,2020-07-07 00
5,200700206,2020-07-05 03:11:38.629000+05:30,2020-07-05 03:11:38.629000+05:30,Dopamine,TYPE0004,0,1,2020-07-05 03


In [73]:
s2

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,rbs,skintemp,centraltemp,temp,day,visitdate,visittime,currentdateweight,currentdateheight,hour
0,0,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,NaN,NaN,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13
1,1,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,NaN,NaN,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13
2,2,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,NaN,NaN,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13
3,3,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,NaN,NaN,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13
4,4,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,NaN,NaN,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,1520,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,36.2,36.2,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14
1521,1521,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,36.2,36.2,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14
1522,1522,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,36.2,36.2,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14
1523,1523,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,36.2,36.2,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14


In [390]:
s2

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,rbs,skintemp,centraltemp,temp,day,visitdate,visittime,currentdateweight,currentdateheight,hour
0,0,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,None,None,None,2020-07-01,2020-07-01,04:07:00,1390.0,39.0,2020-07-01 04
1,1,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,None,None,None,2020-07-01,2020-07-01,04:07:00,1390.0,39.0,2020-07-01 04
2,2,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,None,None,None,2020-07-01,2020-07-01,04:07:00,1390.0,39.0,2020-07-01 04
3,3,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,None,None,None,2020-07-01,2020-07-01,04:07:00,1390.0,39.0,2020-07-01 04
4,4,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,None,None,None,2020-07-01,2020-07-01,04:07:00,1390.0,39.0,2020-07-01 04
5,5,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,None,None,None,2020-07-01,2020-07-01,04:07:00,1390.0,39.0,2020-07-01 04
6,6,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,None,None,None,2020-07-01,2020-07-01,04:07:00,1390.0,39.0,2020-07-01 04
7,7,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,None,None,None,2020-07-01,2020-07-01,04:07:00,1390.0,39.0,2020-07-01 04
8,8,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,None,None,None,2020-07-01,2020-07-01,04:07:00,1390.0,39.0,2020-07-01 04
9,9,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaN,None,None,None,2020-07-01,2020-07-01,04:07:00,1390.0,39.0,2020-07-01 04


In [391]:
meds_dummies

,uhid,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,typevalue_Inotropes,hour
0,200700206,2020-07-05 12:55:36.990000+05:30,2020-07-05 12:55:36.990000+05:30,ZONAMAX-PS,TYPE0001,1,0,2020-07-05 12
1,200700148,2020-07-04 04:22:23.692000+05:30,2020-07-04 04:22:23.692000+05:30,Dopamine,TYPE0004,0,1,2020-07-04 04
2,200700207,2020-07-05 12:43:25.410000+05:30,2020-07-05 12:43:25.410000+05:30,ZONAMAX-PS,TYPE0001,1,0,2020-07-05 12
3,200700148,2020-07-04 04:20:49.535000+05:30,2020-07-04 04:20:49.535000+05:30,ZONAMAX-PS,TYPE0001,1,0,2020-07-04 04
4,200700160,2020-07-07 00:18:05.437000+05:30,2020-07-07 00:18:05.437000+05:30,ZONAMAX-PS,TYPE0001,1,0,2020-07-07 00
5,200700206,2020-07-05 03:11:38.629000+05:30,2020-07-05 03:11:38.629000+05:30,Dopamine,TYPE0004,0,1,2020-07-05 03


In [392]:
meds_dummies['uhid'] = meds_dummies['uhid'].apply(to_float)

In [393]:
s3 = pd.merge(s2,meds_dummies,on = ['uhid','hour'], how='left')

In [394]:
s3['day'] = s3['hour_series'].apply(split_date_1)

In [395]:
s3

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,visittime,currentdateweight,currentdateheight,hour,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,typevalue_Inotropes
0,0,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,04:07:00,1390.0,39.0,2020-07-01 04,NaT,NaT,NaN,NaN,NaN,NaN
1,1,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,04:07:00,1390.0,39.0,2020-07-01 04,NaT,NaT,NaN,NaN,NaN,NaN
2,2,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,04:07:00,1390.0,39.0,2020-07-01 04,NaT,NaT,NaN,NaN,NaN,NaN
3,3,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,04:07:00,1390.0,39.0,2020-07-01 04,NaT,NaT,NaN,NaN,NaN,NaN
4,4,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,04:07:00,1390.0,39.0,2020-07-01 04,NaT,NaT,NaN,NaN,NaN,NaN
5,5,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,04:07:00,1390.0,39.0,2020-07-01 04,NaT,NaT,NaN,NaN,NaN,NaN
6,6,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,04:07:00,1390.0,39.0,2020-07-01 04,NaT,NaT,NaN,NaN,NaN,NaN
7,7,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,04:07:00,1390.0,39.0,2020-07-01 04,NaT,NaT,NaN,NaN,NaN,NaN
8,8,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,04:07:00,1390.0,39.0,2020-07-01 04,NaT,NaT,NaN,NaN,NaN,NaN
9,9,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,04:07:00,1390.0,39.0,2020-07-01 04,NaT,NaT,NaN,NaN,NaN,NaN


In [396]:
#PN
cur8= con.cursor()
cur8.execute("SELECT t1.uhid,t1.entrydatetime,t1.totalparenteralvolume,t1.total_intake FROM kalawati.babyfeed_detail AS t1 where t1.uhid IN "+uhidPatient+";")
cols8 = list(map(lambda x: x[0], cur8.description))
pn = pd.DataFrame(cur8.fetchall(),columns=cols8)

In [201]:
len(pn)

32

In [397]:
pn['hour'] = pn.entrydatetime.apply(split_hour)
pn['total_intake'] = pn['total_intake'].astype(float)
pn['tpn-tfl'] = pn['totalparenteralvolume']/pn['total_intake']

In [398]:
pn['uhid'] = pn['uhid'].apply(to_float)

In [399]:
s4 = pd.merge(s3,pn,on = ['uhid','hour'], how='left')

In [400]:
s4

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,typevalue_Inotropes,entrydatetime,totalparenteralvolume,total_intake,tpn-tfl
0,0,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,1,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,2,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,3,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,4,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
5,5,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
6,6,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
7,7,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
8,8,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
9,9,2020-07-01 04:07:00,200700003.0,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,...,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [81]:
len(s4.uhid.unique())

1

In [401]:
s4.drop_duplicates(subset=['uhid','hour_series'],inplace=True)

In [402]:
q = pd.DataFrame()

In [403]:
for i in s4.uhid.unique():
    x = s4[s4['uhid']==i]
    x['currentdateheight'].fillna(method='ffill',inplace=True)
    x['currentdateweight'].fillna(method='ffill',inplace=True)
    q = q.append(x,ignore_index=True)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [404]:
def abd(x):
    
    try:   
        
        if x>200:
            return x/10
        else:
            return x
    except:
        pass
    

In [405]:
q['abdomen_girth'] = pd.to_numeric(q['abdomen_girth'], errors='coerce')
q['abdomen_girth'] = q['abdomen_girth'].apply(abd)

In [406]:
dt = pd.DataFrame()
for i in q.uhid.unique():
    x = q[q['uhid']==i]
    #for j in y['day'].unique():
    #x = y[y['day']==j]
    x['abdomen_girth'].fillna(method='bfill',inplace=True)
    x['currentdateweight'].fillna(method='bfill',inplace=True)
    x['currentdateheight'].fillna(method='bfill',inplace=True)
    x['totalparenteralvolume'].fillna(method='ffill',inplace=True)
    x['total_intake'].fillna(method='ffill',inplace=True)
    x['tpn-tfl'].fillna(method='ffill',inplace=True)
    dt = dt.append(x,ignore_index=True)

In [407]:
dt.urine_per_hour.max()

inf

In [89]:
dt.urine_per_hour.unique()

array([1.21971786,        inf, 6.26021502])

In [408]:
a = dt.replace([np.inf, -np.inf], np.nan)

In [409]:
def abd_2(x):
    if not(x is None) and x < 0:
        return ((-1)*x)/10
    elif not(x is None) and x > 200:
        return x/10
    else:
        return x
    
a['abdomen_girth'] = a['abdomen_girth'].apply(abd_2)

def abd_3(x):
    if not(x is None) and x < 15:
        return np.nan
    elif not(x is None) and x > 50:
        return np.nan
    else:
        return x
    
a['abdomen_girth'] = a['abdomen_girth'].apply(abd_3)

def upm(x):
    if not(x is None) and x > 100:
        return np.nan
    else:
        return x
    
a['urine_per_hour'] = a['urine_per_hour'].apply(upm)

def temp_f_c(x):
    if not(x is None) and x > 90:
        return (x - 32)*(5/9)
    else:
        return x
    
a['temp'] = a['temp'].apply(temp_f_c)

def weight_correct(x):
    if not(x is None) and x<10:
        return x*100
    elif x>5000:
        return x/10
    else:
        return x
    
def weight_correct_2(x):
    if not(x is None) and x<300:
        return x*10
    elif x>5000:
        return x/10
    else:
        return x
    
a.currentdate = a.currentdateweight.apply(weight_correct)
a.currentdate = a.currentdateweight.apply(weight_correct_2)

def height_correct(x):
    if not(x is None) and x < 10:
        return np.nan
    else:
        return x
    
def rbs_correct(x):
    if not(x is None) and x > 700:
        return np.nan
    else:
        return x
    
a['currentdateheight'] = a['currentdateheight'].apply(height_correct)

a['rbs'] = a['rbs'].apply(rbs_correct)



/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [410]:
len(a.uhid.unique())

5

In [411]:
data = a.copy()

In [412]:
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass

In [413]:
data['day'] = data['hour_series'].apply(split_date_1)

In [414]:
def to_date(x):
    return pd.to_datetime(x)

In [415]:
data['hour_series'] = data['hour_series'].apply(to_date)

In [218]:
len(data.uhid.unique())

5

In [416]:
data.to_csv("intermittent_checkpoint_2.csv")

In [417]:
df = pd.DataFrame()
for i in data.uhid.unique():
    x = data[data['uhid']==i]
    x.rbs.fillna(method='ffill',inplace=True)
    x.currentdateweight.fillna(method='ffill',inplace=True)
    x.currentdateheight.fillna(method='ffill',inplace=True)
    n = math.ceil(len(x)/24)+1
    start_date = pd.to_datetime(x['day'].iloc[0]+" " + "08:00:00") - timedelta(hours=24)
    for i in range(int(n)):
        
       
        y = x[(x['hour_series']>=start_date + timedelta(hours=24*i)) & (x['hour_series']<=start_date + timedelta(hours=24*(i+1)))]
        y['stool_day_total'] = y['stool_passed'].sum()
        df =df.append(y,ignore_index=True)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [101]:
df.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_3.csv')

In [418]:
test = df.copy()

In [419]:
dq = pd.DataFrame()

In [420]:
for i in test.uhid.unique():
    x = test[test['uhid']==i]
    x.abdomen_girth.fillna(method='ffill',inplace=True)
    dq = dq.append(x, ignore_index=True)

In [421]:
final = pd.DataFrame()

In [422]:
for i in dq.uhid.unique():
    x = dq[dq['uhid']==i]
    x['abd_difference_y'] = x['abdomen_girth'] - x['abdomen_girth'].shift(periods=1)
    final = final.append(x,ignore_index=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [107]:
final.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_4.csv')

In [423]:
cur1 = con.cursor()
cur1.execute("SELECT t1.uhid,t1.starttime,t1.pulserate, t1.ecg_resprate, t1.spo2, t1.heartrate, t2.dischargestatus,t1.mean_bp,t1.sys_bp,t1.dia_bp  FROM kalawati.device_monitor_detail AS t1 LEFT JOIN kalawati.baby_detail AS t2 ON t1.uhid=t2.uhid WHERE (t2.uhid IN "+ uhidPatient+") OR (t2.uhid IN "+ uhidPatient+");")
cols1 = list(map(lambda x: x[0], cur1.description))
ds = pd.DataFrame(cur1.fetchall(),columns=cols1)
ds.sort_values('starttime',inplace=True)

In [243]:
len(ds)

350210

In [424]:
ds.spo2.unique()

array(['98', '100', '95', '94', '93', '96', '92', '97', '91', None, '99',
       '88', '85', '83', '86', '90', '80', '69', '72', '67', '61', '58',
       '56', '87', '89', '82', '84', '65', '75', '79', '76', '78', '73',
       '55', '66', '63', '62', '64', '59', '77', '31', '38', '81', '74'],
      dtype=object)

In [425]:
test = ds.drop_duplicates(subset=['uhid','starttime'],keep='first')

In [426]:
ds.spo2.unique()

array(['98', '100', '95', '94', '93', '96', '92', '97', '91', None, '99',
       '88', '85', '83', '86', '90', '80', '69', '72', '67', '61', '58',
       '56', '87', '89', '82', '84', '65', '75', '79', '76', '78', '73',
       '55', '66', '63', '62', '64', '59', '77', '31', '38', '81', '74'],
      dtype=object)

In [427]:
cur_vent = con.cursor()
cur_vent.execute("SELECT t1.uhid,t1.start_time,t1.creationtime,t1.peep, t1.pip,t1.map ,t1.tidalvol, t1.minvol,t1.ti,t1.fio2 FROM kalawati.device_ventilator_detail AS t1 LEFT JOIN kalawati.baby_detail AS t2 ON t1.uhid=t2.uhid WHERE  t2.uhid IN "+uhidPatient+";")
cols_vent = list(map(lambda x: x[0], cur_vent.description))
ventilator_cont = pd.DataFrame(cur_vent.fetchall(),columns=cols_vent)

test_vent = ventilator_cont.drop_duplicates(subset=['uhid','start_time'],keep='first')

def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

test['date'] = test.starttime.apply(split_date)
test_vent['date'] = test_vent.start_time.apply(split_date)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [428]:
cont_data = pd.merge(test,test_vent,on=['uhid','date'],how='left',copy=False)
test_cont = cont_data.drop_duplicates(subset=['uhid','starttime','heartrate'],keep='first')


In [271]:
ds.spo2.unique()

array(['98', '100', '95', '94', '93', '96', '92', '97', '91', None, '99',
       '88', '85', '83', '86', '90', '80', '69', '72', '67', '61', '58',
       '56', '87', '89', '82', '84', '65', '75', '79', '76', '78', '73',
       '55', '66', '63', '62', '64', '59', '77', '31', '38', '81', '74'],
      dtype=object)

In [274]:
test_cont.hour_series

0        2020-07-03 17
1        2020-07-03 17
2        2020-07-04 10
3        2020-07-04 10
4        2020-07-04 10
5        2020-07-04 10
6        2020-07-04 10
7        2020-07-04 10
8        2020-07-04 10
9        2020-07-04 10
10       2020-07-04 10
11       2020-07-04 10
12       2020-07-04 10
13       2020-07-04 10
14       2020-07-04 10
15       2020-07-04 10
16       2020-07-04 10
17       2020-07-04 10
18       2020-07-04 10
19       2020-07-04 10
20       2020-07-04 10
21       2020-07-04 10
22       2020-07-04 10
23       2020-07-04 10
24       2020-07-04 10
25       2020-07-04 10
26       2020-07-04 10
27       2020-07-04 10
28       2020-07-04 10
29       2020-07-04 10
             ...      
54366    2020-07-09 10
54367    2020-07-09 10
54368    2020-07-09 10
54369    2020-07-09 10
54370    2020-07-09 10
54371    2020-07-09 10
54372    2020-07-09 10
54373    2020-07-09 10
54374    2020-07-09 10
54375    2020-07-09 10
54376    2020-07-09 10
54377    2020-07-09 10
54378    20

In [429]:
test_cont['hour_series'] = test_cont['date'].apply(split_hour)

In [430]:
def to_str(x):
    return str(x)

In [431]:
test_cont['ref_hour'] = test_cont['hour_series'].apply(to_str)
final['ref_hour'] = final['hour_series'].apply(to_str)

In [432]:
ds.spo2.unique()

array(['98', '100', '95', '94', '93', '96', '92', '97', '91', None, '99',
       '88', '85', '83', '86', '90', '80', '69', '72', '67', '61', '58',
       '56', '87', '89', '82', '84', '65', '75', '79', '76', '78', '73',
       '55', '66', '63', '62', '64', '59', '77', '31', '38', '81', '74'],
      dtype=object)

In [433]:
def con_time(x):
    return str(x).split("+")[0].split(":")[0] + ":" +str(x).split("+")[0].split(":")[1]

In [434]:
test_cont['cont_time'] = test_cont.starttime.apply(con_time)

In [256]:
ds.spo2.unique()

array(['98', '100', '95', '94', '93', '96', '92', '97', '91', None, '99',
       '88', '85', '83', '86', '90', '80', '69', '72', '67', '61', '58',
       '56', '87', '89', '82', '84', '65', '75', '79', '76', '78', '73',
       '55', '66', '63', '62', '64', '59', '77', '31', '38', '81', '74'],
      dtype=object)

In [435]:
def con_time_2(x):
    return str(x).split(":")[0] +":"+str(x).split(":")[1]

In [441]:
test_cont['uhid'] = test_cont['uhid'].apply(to_float)

In [440]:
test_cont['cont_time'] = test_cont.ref_hour.apply(con_time_2)

IndexError: list index out of range

In [284]:
test_cont_1 = test_cont[test_cont.uhid=="200700003"]

In [ ]:
final_1 = final[final.uhid=="200700003"]

In [439]:
test_cont

,uhid,starttime,pulserate,ecg_resprate,spo2,heartrate,dischargestatus,mean_bp,sys_bp,dia_bp,...,peep,pip,map,tidalvol,minvol,ti,fio2,hour_series,ref_hour,cont_time
0,200700206,2020-07-03 17:31:04+05:30,None,51,98,150,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-03 17,2020-07-03 17,2020-07-03 17:31
1,200700206,2020-07-03 17:31:35+05:30,None,51,100,146,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-03 17,2020-07-03 17,2020-07-03 17:31
2,200700148,2020-07-04 10:19:09+05:30,174,45,95,175,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-04 10,2020-07-04 10,2020-07-04 10:19
3,200700148,2020-07-04 10:20:09+05:30,174,45,95,174,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-04 10,2020-07-04 10,2020-07-04 10:20
4,200700148,2020-07-04 10:21:09+05:30,174,45,94,174,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-04 10,2020-07-04 10,2020-07-04 10:21
5,200700148,2020-07-04 10:22:09+05:30,174,45,93,174,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-04 10,2020-07-04 10,2020-07-04 10:22
6,200700148,2020-07-04 10:23:09+05:30,174,43,93,174,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-04 10,2020-07-04 10,2020-07-04 10:23
7,200700148,2020-07-04 10:24:09+05:30,174,43,96,174,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-04 10,2020-07-04 10,2020-07-04 10:24
8,200700148,2020-07-04 10:25:09+05:30,174,44,93,174,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-04 10,2020-07-04 10,2020-07-04 10:25
9,200700148,2020-07-04 10:26:09+05:30,174,44,93,174,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-04 10,2020-07-04 10,2020-07-04 10:26


In [442]:
qw = pd.merge(final,test_cont, on=['uhid','cont_time'],how='left')

In [121]:
#final[final['cont_time']=='2019-09-30 11:18']

In [122]:
#qw.dropna(subset=['series'],inplace=True)

In [444]:
qw.to_csv('intermediate_checkpoint_apollo_5_9july_final.csv')

In [443]:
qw.spo2.unique()

array([nan, '99', '98', '95', '96', '94', '97', '90', '93', '91', '92',
       None, '100', '82', '88', '89', '87', '81', '85', '74', '75', '86',
       '73', '83', '79', '77', '84', '80', '69', '72', '67', '61', '58',
       '56', '65', '76', '78', '55', '66', '63', '62', '64', '59', '31',
       '38'], dtype=object)

In [239]:
qw.count()

Unnamed: 0                  41434
actual_DOB                  41434
uhid                        41434
dischargeddate              41434
actual_DOA                  41434
gender                      41434
birthweight                 41434
birthlength                 41434
birthheadcircumference      41434
inout_patient_status        41434
gestationweekbylmp          41434
gestationdaysbylmp          41434
baby_type                   41434
central_temp                14873
apgar_onemin                36611
apgar_fivemin               41434
apgar_tenmin                41434
motherage                       0
conception_type             27514
mode_of_delivery            41434
steroidname                 20077
numberofdose                12641
dischargestatus_x               0
birthlength.1               41434
birthheadcircumference.1    41434
inout_patient_status.1      41434
gestationweekbylmp.1        41434
gestationdaysbylmp.1        41434
baby_type.1                 41434
central_temp.1